In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

loader = DirectoryLoader('C:\\Users\\amine\\chatbot\\cases\\', glob="*case.txt",show_progress=True,loader_cls=TextLoader)
data = loader.load()
data
doc_sources = [doc.metadata['source']  for doc in data]
doc_sources

100%|██████████| 20/20 [00:00<00:00, 78.34it/s]


['C:\\Users\\amine\\chatbot\\cases\\c10case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c11case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c12case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c13case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c14case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c15case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c16case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c17case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c18case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c19case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c1case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c20case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c2case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c3case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c4case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c5case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c6case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c7case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c8case.txt',
 'C:\\Users\\amine\\chatbot\\cases\\c9case.txt']

In [6]:


import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, EmbeddingsFilter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, create_history_aware_retriever, create_retrieval_chain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.chat_models import ChatOllama

import torch
from langchain.schema.output_parser import StrOutputParser

In [7]:
# Split the extracted data into text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300)
text_chunks = text_splitter.split_documents(data)

# Create embeddings for each text chunk
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings).as_retriever(search_kwargs={"k": 1})

In [9]:
model=ChatOllama(model="mistral",
                  temperature=0,
                  num_ctx=32768,
                  top_k=10,
                  top_p=0.5,
                  keep_alive=-1,
                  num_predict=4096,
                  num_batch=1024)

In [10]:
compressor = LLMChainExtractor.from_llm(model)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_store
)

In [11]:
compressor = LLMChainExtractor.from_llm(model)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_store
)
x=compression_retriever.get_relevant_documents("who delivered the opinion of the court in the case of SILVERTHORNE LUMBER COMPANY, INC., ET AL. v. UNITED STATES. No. 358.?")

c:\Users\amine\anaconda3\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [12]:
x

[Document(page_content='MR. JUSTICE HOLMES delivered the opinion of the court.\nThis is a writ of error brought to reverse a judgment of the District Court fining the Silverthorne Lumber Company two hundred and fifty dollars for contempt of court and ordering Frederick W. Silverthorne to be imprisoned until he should purge himself of a similar contempt. The contempt in question was a refusal to obey subpoenas and an order of Court to produce books and documents of the company before the grand jury to be used in regard to alleged violation of the statutes of the United States by the said Silverthorne and his father. One ground of the refusal was that the order of the Court infringed the rights of the parties under the Fourth Amendment of the Constitution of the United States.', metadata={'source': 'C:\\Users\\amine\\chatbot\\cases\\c10case.txt'})]

In [15]:
import re
compressor = LLMChainExtractor.from_llm(model)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_store
)
x=compression_retriever.get_relevant_documents("who delivered the opinion of the court in the case of SILVERTHORNE LUMBER COMPANY, INC., ET AL. v. UNITED STATES. No. 358.?")
metadata = {'source': 'C:\\Users\\amine\\chatbot\\cases\\c10case.txt'}
source = metadata['source']

# Using regular expression to extract 'c' followed by a number
match = re.search(r'c\d+', source)
case_identifier = match.group(0)
print(case_identifier)
# Construct path for the opinion file
opinion_path = os.path.join(os.path.dirname(source), case_identifier + 'opinion.txt')

# Read the opinion from the file
with open(opinion_path, 'r') as opinion_file:
    opinion = opinion_file.read()

print(opinion)

c10


In [17]:
import os
# Construct path for the opinion file
opinion_path = os.path.join(os.path.dirname(source), case_identifier + 'opinion.txt')

# Read the opinion from the file
with open(opinion_path, 'r') as opinion_file:
    opinion = opinion_file.read()

print(opinion)

 Opinion: The Supreme Court held that the Fourth Amendment protection against unlawful searches and seizures extends beyond just the physical possession of the seized items, but also prevents the government from using any knowledge gained through such an unlawful means. In this case, the Silverthorne Lumber Company and its officers refused to comply with subpoenas to produce documents after they had been illegally seized by government officials. The Court ruled that the government could not use the knowledge obtained from the illegal seizure, even if it later returned the originals of the seized materials. The Court reasoned that the essence of the Fourth Amendment is to forbid the acquisition of evidence in an unlawful way, and that this prohibition applies not only to the use of such evidence before a court but also to any subsequent use of the knowledge gained from it. The judgment was reversed, with two dissenting justices.


In [1]:
%pip install --upgrade huggingface_hub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from huggingface_hub import login
login("hf_token")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\amine\.cache\huggingface\token
Login successful


In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("yoru01/modelna")

config.json:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

c:\Users\amine\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amine\.cache\huggingface\hub\models--yoru01--modelna. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

In [4]:
model.save_pretrained('./my_model')

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

c:\Users\amine\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amine\.cache\huggingface\hub\models--distilbert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [6]:
tokenizer.save_pretrained('./my_tokenizer')

('./my_tokenizer\\tokenizer_config.json',
 './my_tokenizer\\special_tokens_map.json',
 './my_tokenizer\\vocab.txt',
 './my_tokenizer\\added_tokens.json',
 './my_tokenizer\\tokenizer.json')

In [14]:
import json
import torch
from transformers import DistilBertForTokenClassification, AutoTokenizer

model = DistilBertForTokenClassification.from_pretrained('C:\\Users\\amine\\chatbot\\src\\my_model')
tokenizer = AutoTokenizer.from_pretrained("C:\\Users\\amine\\chatbot\\src\\my_tokenizer")

# Load the label dictionary
with open('C:\\Users\\amine\\chatbot\\data\\data.json', 'r') as f:
    data = json.load(f)

# Define the input sentence
sentence = input("Please enter a sentence: ")

# Tokenize the sentence and get model predictions
inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
model.eval()
with torch.no_grad():
    logits = model(**inputs).logits

# Convert predictions to labels
predicted_token_ids = torch.argmax(logits, dim=-1).numpy().tolist()[0]
id_to_label = {id: label for label, id in data.items()}
predicted_labels = [id_to_label[id] for id in predicted_token_ids]

# Remove the last two labels (corresponding to [SEP] and [PAD] tokens)
predicted_labels = predicted_labels[:-2]

# Merge consecutive labels of the same type
merged_labels = []
current_label = None
for label in predicted_labels:
    if label == 'O' or label != current_label:
        merged_labels.append(label)
        current_label = label
    else:
        merged_labels[-1] = label

# Pair each word with its label
output_words = []
for word, label in zip(sentence.split(), merged_labels):
    if label == 'O':
        output_words.append(word)
    else:
        output_words.append(label[2:])

# Join the output words into a sentence
output_sentence = " ".join(output_words)

print(output_sentence)



my name is FIRSTNAME i am AGE AGE old and i work as JOBTYPE Engineer, i am from STATE i came to STATE to visit a friend, i entred in a forbidden land in San Diego and CITY CITY to know what possibly could be my punishement ?


In [1]:
import json
import os

# Load JSON file
with open('C:\\Users\\amine\\output5.json', 'r') as json_file:
    data = json.load(json_file)

# Create a directory to store text files
if not os.path.exists('cases'):
    os.makedirs('cases')

# Iterate through cases
for case_name, case_data in data['cases'].items():
    case_content = case_data['case']
    opinion_content = case_data['opinion']
    
    # Write case content to file
    with open(f'cases/{case_name}case.txt', 'w') as case_file:
        case_file.write(case_content)
    
    # Write opinion content to file
    with open(f'cases/{case_name}opinion.txt', 'w') as opinion_file:
        opinion_file.write(opinion_content)


In [1]:
# Importing the os module to work with file paths and directories
import os

# Defining a function to extract paragraphs from a text based on a specific delimiter
def extract_paragraphs(text):
    # Initializing an empty list to store the extracted paragraphs
    paragraphs = []
    # Initializing the starting index for searching for the delimiter
    start_index = 0
    # Looping indefinitely until the end of the text is reached
    while True:
        # Finding the starting index of the delimiter "~~~~" in the text
        start_index = text.find("~~~~", start_index)
        # If the delimiter is not found, exit the loop
        if start_index == -1:
            break
        # Finding the ending index of the delimiter "~~~~" in the text
        end_index = text.find("~~~~", start_index + 1)
        # If the delimiter is not found, exit the loop
        if end_index == -1:
            break
        # Extracting the paragraph between the delimiters and stripping whitespace
        paragraph = text[start_index + len("~~~~"):end_index].strip()
        # Appending the extracted paragraph to the list of paragraphs
        paragraphs.append(paragraph)
        # Updating the start index for the next iteration
        start_index = end_index + len("~~~~")
    # Returning the list of extracted paragraphs
    return paragraphs

# Defining a function to process a file, extract paragraphs, and write them to output files
def process_file(file_path, output_directory):
    # Opening the file in read mode
    with open(file_path, 'r') as file:
        # Reading the entire contents of the file into a string variable
        text = file.read()
        # Extracting paragraphs from the text using the defined function
        paragraphs = extract_paragraphs(text)
        # Extracting the header from the text between the delimiters "****"
        header = text[text.find("****") + len("****"):text.find("****", text.find("****") + 1)].strip()
        # Iterating over each extracted paragraph
        for i, paragraph in enumerate(paragraphs):
            # Generating the output file path based on the input file name and index
            output_file_path = os.path.join(output_directory, f"{os.path.splitext(os.path.basename(file_path))[0]}_{i}.txt")
            # Opening the output file in write mode
            with open(output_file_path, 'w') as output_file:
                # Writing the header, paragraph, and a newline character to the output file
                output_file.write(header + '\n\n' + paragraph)

# Defining a function to process all files in a directory recursively
def process_directory(input_directory, output_directory):
    # Walking through the directory tree rooted at the input directory
    for root, _, files in os.walk(input_directory):
        # Iterating over each file found in the directory
        for file in files:
            # Checking if the file has a ".txt" extension
            if file.endswith(".txt"):
                # Constructing the full file path
                file_path = os.path.join(root, file)
                # Calling the process_file function to process the current file
                process_file(file_path, output_directory)

# Setting the input directory path where the text files are located
input_directory_path = "C:\\Users\\amine\\chatbot\\testfile"
# Setting the output directory path where the processed files will be written
output_directory_path = "C:\\Users\\amine\\chatbot\\testfile2"

# Calling the process_directory function to process all files in the input directory
# and write the processed files to the output directory
process_directory(input_directory_path, output_directory_path)
